In [1]:
path = 'REC NOT CLOSED VFinale2.xlsm'
import pandas as pd

In [5]:
def preprocess_data(path = '../RESOURCE SUMMARY_20220912.xlsx'):
    wb = pd.read_excel(path,'Sheet1',header=1)

    correspondences = {}

    nonworking = ['10th Day','9th Day','8th day','All Saints Day','Annual Leave','Ascension Day','Easter Monday','National Day',
    'Other Leave','Sick Leave','Special Leave granted by the DG','Whit Monday']

    non_audit = ['Administrative Matters & Support','Covid-19 Sanitary Situation','GC and EXB (incl. Annual report)',
        'HR Management & Recruitment', 'IOS Management - Ad-hoc Requests','IOS Team Meetings', 'JIU Coordination',
        'OAC support, preparation and meetings','Participation to UNESCO Working Groups or Task Forces',
        'Policy or Administrative Manual Item Review','Support to Investigation Unit',
        'Trainings or Workshops']

    audit = ['Audit-Ad-hoc request / Advisory','Audit-Annual Planning', 'Audit-QAIP (incl. TeamMate+ Migration)']

    reco = ['Audit-Recommendation Follow-up']

    correspondences['Nonworking'] = nonworking
    correspondences['Non audit time'] = non_audit
    correspondences['Audit Time'] = audit
    correspondences['Reco Follow-up Time'] = reco

    # Insert correspondences for the different Time Categories
    wb['Reporting categories'] = 'Audit Time'
    for i in correspondences: 
        wb['Reporting categories'].mask(wb['Time Category'].isin(correspondences[i]),i,inplace=True)

    # Make Phase 5 -- Recommendation followups be repored as Reco Follow up time
    wb['Reporting categories'].mask(wb['Phase'].astype('string').str.startswith('5'),'Reco Follow-up Time',inplace=True)

    # Remove IOS/... prefix from Assignment name
    prefix_removed = wb['Assignment Name'].str.extract(r'(^IOS/[a-zA-Z0-9_\./]+)-([a-zA-Z0-9_\' -]+)')[1]
    idxs = ~wb['Assignment Name'].str.startswith('IOS/').fillna(False)
    prefix_removed[idxs] = wb[idxs]['Assignment Name']
    wb['Assignment Name'] = prefix_removed

    offices = ['Addis','IICBA','Harare','Havana','Yaound','San Jos','Tehran']
    sheet_names = ['IRA_Addis_20220616_after CSA','IRA IICBA', 'IRA Harare 24 June 2022', 'IRA Havana Final', 'IRA Yaounde',
            'San José_Leadsheet ', 'IRA Teheran_Jan 2022']
    connector = dict(zip(offices,sheet_names))

    actual = wb[wb['Assignment Name'].str.contains('|'.join(offices),na=False,case=False)]


    return wb

In [6]:
wb = preprocess_data()

In [7]:
with pd.ExcelWriter('resource_data_cleaned.xlsx',mode = 'w',engine='openpyxl') as writer:
    wb.to_excel(writer,sheet_name='sheet_1',index=False)

In [16]:
wb[wb['Assignment Code'].fillna('').str.contains('FO')]['Assignment Code'].str.slice(5).value_counts()

FO-San Jose               258
FO-Crosscutting Issues    254
FO-Yaounde                245
FO-TEH                    191
FO-MXC                    177
FO-Harare                 174
FO-Addis Ababa            163
FO-DHA                    124
FO-KAT                    121
FO-HAV                    116
FO-JUB                     75
FO-Vendors & Bank          17
FO-Kinshasa Remote          6
Name: Assignment Code, dtype: int64